<a href="https://colab.research.google.com/github/ShanGemunu/Code-Unity-News-Analysis/blob/news-analysis-using-JSON/TestNewsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install investpy
#!pip install python-dateutil

import pandas as pd
import investpy









invest = investpy.news.economic_calendar(time_zone="GMT +5:30")  #  ,from_date="15/11/2022", to_date="20/11/2022"

#USD Events       ,  note --- dataframe.iloc[:,3] -> all row, particular column    and   dataframe.iloc[3,4] -> 4th row, 5th column
usdEvents = (invest[invest.iloc[:, 4] == "USD"]).reset_index()  # ---------- (01)    ,   this usdEvents is new dataframe, it is using usual indexs as 0,1,2,3,....

print(usdEvents.to_string())
print("---------------")

def removeLowImpacts():      # to remove events having "low" impact  ------------(02)
  df = pd.DataFrame()   # create new dataframe
  for x in range(0,len(usdEvents.index)):
    if(usdEvents.iloc[x].importance=="medium" or usdEvents.iloc[x].importance=="high"):   # usdEvents.iloc[x] means object & importance is like attribute
       df = df.append(usdEvents.iloc[x,:], ignore_index=True)
  
  return df 

def detectTrendSignal(dataframe):  #to detect trend signals   ----------------(03)(a)
  dataframe['signal'] = None
  for x in range(0,len(dataframe.index)):
    if(dataframe.iloc[x].forecast==None or dataframe.iloc[x].previous==None):
      dataframe.iloc[x,11] = None
    else:
      forecast = dataframe.iloc[x].forecast
      previous = dataframe.iloc[x].previous

      forecast = forecast.replace('M','')
      forecast = forecast.replace('%','')
      forecast = forecast.replace('K','')
      forecast = forecast.replace('B','')
      forecast = float(forecast)

      previous = previous.replace('M','')
      previous = previous.replace('%','')
      previous = previous.replace('K','')
      previous = previous.replace('B','')
      previous = float(previous)

      if(forecast<previous):
        dataframe.iloc[x,11] = "Buy"
      elif(previous<forecast):
        dataframe.iloc[x,11] = "Sell"
      else:
        dataframe.iloc[x,11] = "Neutral"  

  return dataframe

def separateGroups(dataframe):   # to separate groups  ---------------(03)(b)          
  y = 0
  groupList = [pd.DataFrame()]
  groupList[0] = groupList[0].append(dataframe.iloc[0,:], ignore_index=True)
  for x in range(1,len(dataframe.index)):
    if(dataframe.iloc[x,11]==dataframe.iloc[x-1,11]):
      groupList[y] = groupList[y].append(dataframe.iloc[x,:], ignore_index=True)   
    else:
      groupList.append(pd.DataFrame())
      y += 1
      groupList[y] = groupList[y].append(dataframe.iloc[x,:], ignore_index=True)
  return groupList     # return list contains dataframes

def findHighImpacts(list):    # check whether previous groups have at least one high impact event & separate those groups ----------------(04) & (05)
  groupList = []
  for i in list:
    for x in range(0,len(i.index)):
      if(i.iloc[x,6]=="high"):
        groupList.append(i)
        break
  return groupList    # return list contains dataframes  

def generateTimeIntervals(list):    # generate time/time interval of those groups made in previous step   ------------------------(06)
  df = pd.DataFrame(columns=['from_time','to_time','signal']) 
  for i in list:
    array =[]   # create new array as new row to "df" dataframe
    from_time = None
    to_time = None
    # dic_ =	{}   
    # dic_["signal"] = i.iloc[0,11]
    if(len(i.index)==1):
      from_time = i.iloc[0,2] +" "+ i.iloc[0,3]
    else:  
      from_time = i.iloc[0,2] +" "+ i.iloc[0,3]
      to_time = i.iloc[len(i.index)-1,2] +" "+ i.iloc[len(i.index)-1,3]

    array.append(from_time)
    array.append(to_time)
    array.append(i.iloc[0,11])
    df.loc[len(df.index)] = array  # add new row into "df" dataframe
    # df = df.append(dic_, ignore_index=True)    add new row into "df" dataframe
  
  return df    # return dataframe having time/time intervals and signal



dataframeList_last = pd.DataFrame()

if(len(usdEvents.index)!=0):
  dataframe_02 = removeLowImpacts()  # output of step 02
  if(len(dataframe_02.index)!=0):
    dataframe_03 = detectTrendSignal(dataframe_02)   # output of step 03-a
    
    if(len(dataframe_03.index)!=0):
      dataframeList_01 = separateGroups(dataframe_03)   # output of step 03-b  , It,s a list of dataframes
      print(dataframeList_01)
      print("---------------")
      if(len(dataframeList_01) != 0):
        dataframeList_02 = findHighImpacts(dataframeList_01)   # output of step 4 & 5,  it's a list of dataframes
        if(len(dataframeList_02) != 0):
          dataframeList_last = generateTimeIntervals(dataframeList_02)  # output of step 6, it,s dataframe having time/time intervals and signal
        else:
          print("Nothing,,, code execution is stopped.")  
      else:
        print("Nothing,,, code execution is stopped.")

    else:
      print("Nothing,,, code execution is stopped.")
  else:
    print("Nothing,,, code execution is stopped.")
  
else:
  print("Nothing,,, code execution is stopped.")


json = dataframeList_last.to_json()
print(dataframeList_last.to_string()) 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 31.8 MB/s 
     |████████████████████████████████| 235 kB 41.8 MB/s 
  Created wheel for investpy: filename=investpy-1.0.8-py3-none-any.whl size=4481592 sha256=a5eb99694f5bd71a9071da41d6634ac55865872087fdaa14014cd5746ab3bdbb
  Stored in directory: /root/.cache/pip/wheels/9a/6f/8e/21466569113166e5859029b97fcae2b81af57063ed9faf698e
Successfully built investpy
    index      id        date   time           zone currency importance                                 event actual forecast previous
0      48  463672  22/12/2022  19:00  united states      USD        low  Chicago Fed National Activity  (Nov)   None     None    -0.05
1      49  464434  22/12/2022  19:00  united states      USD        low             Continuing Jobless Claims   None   1,683K   1,671K
2      50  463669  22/12/2022  19:00  united states      USD        low                 C

In [5]:
# import urllib library
from urllib.request import urlopen
import pandas as pd
import json
import re
from pandas import json_normalize
  
# import json                               note --- dataframe.iloc[:,3] -> all row, particular column    and   dataframe.iloc[3,4] -> 4th row, 5th column
import json        
# store the URL in url as 
# parameter for urlopen
url = "https://economic-calendar.deta.dev/"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
# print(data_json)

dataframe = json_normalize(data_json)   #  -------- dataframe having usdevents     ------------(01)

dataframe = dataframe[ (dataframe['impact'] == "medium") | (dataframe['impact'] == "high")]     # to remove events having "low" impact  ------------(02)

print(dataframe.to_string())

def detectTrendSignal():  #to detect trend signals   ----------------(03)(a)
  dataframe['signal'] = None

  for x in range(0,len(dataframe.index)):
    if((dataframe.iloc[x, 4] != "") and (dataframe.iloc[x, 5] != "")):
      previous = re.findall(r"[-+]?(?:\d*\.*\d+)", dataframe.iloc[x, 4])   #  value in string type
      consensus = re.findall(r"[-+]?(?:\d*\.*\d+)", dataframe.iloc[x, 5])  #  value in string type

      previous = float(previous[0])   #  value in float type
      consensus = float(consensus[0])   #  value in float type

      if(consensus<previous):
        dataframe.iloc[x,7] = "Buy"
      elif(previous<consensus):
        dataframe.iloc[x,7] = "Sell"
      else:
        dataframe.iloc[x,7] = "Neutral"
      
    else:
      dataframe.iloc[x, 7] = None  


detectTrendSignal()     #  calling to (03)(a) 

def separateGroups(dataframe_):   # to separate groups  ---------------(03)(b)          
  y = 0
  groupList = [pd.DataFrame()]
  groupList[0] = groupList[0].append(dataframe_.iloc[0,:], ignore_index=True)
  for x in range(1,len(dataframe_.index)):
    if(dataframe_.iloc[x,7]==dataframe_.iloc[x-1,7]):
      groupList[y] = groupList[y].append(dataframe_.iloc[x,:], ignore_index=True)   
    else:
      groupList.append(pd.DataFrame())
      y += 1
      groupList[y] = groupList[y].append(dataframe_.iloc[x,:], ignore_index=True)
  return groupList     # return list contains dataframes


def findHighImpacts(list):    # check whether previous groups have at least one high impact event & separate those groups ----------------(04) & (05)
  groupList = []
  for i in list:
    for x in range(0,len(i.index)):
      if(i.iloc[x,3]=="high"):
        groupList.append(i)
        break
  return groupList    # return list contains dataframes  


def generateTimeIntervals(list):    # generate time/time interval of those groups made in previous step   ------------------------(06)
  df = pd.DataFrame(columns=['from_time','to_time','signal']) 
  for i in list:
    array =[]   # create new array as new row to "df" dataframe
    from_time = None
    to_time = None
    # dic_ =	{}   
    # dic_["signal"] = i.iloc[0,11]
    if(len(i.index)==1):
      from_time = i.iloc[0,2]
    else:  
      from_time = i.iloc[0,2]
      to_time = i.iloc[len(i.index)-1,2]

    array.append(from_time)
    array.append(to_time)
    array.append(i.iloc[0,7])
    df.loc[len(df.index)] = array  # add new row into "df" dataframe
    # df = df.append(dic_, ignore_index=True)    add new row into "df" dataframe
  
  return df    # return dataframe having time/time intervals and signal



groupList_01 = separateGroups(dataframe)  # output of (03)(b) , list contains dataframes

groupList_02 = findHighImpacts(groupList_01)  #  output of (04) & (05) , list contains dataframes

dataframeLastOutput = generateTimeIntervals(groupList_02)   #  output of (06)

print(dataframeLastOutput)








                                  title                                                                                                link                     pubDate  impact   previous consensus     actual
13                      Current Account                     https://www.myfxbook.com/forex-economic-calendar/united-kingdom/current-account  Thu, 22 Dec 2022 07:00 GMT  medium  £-33.768B  £-20.84B  £-19.402B
14                  GDP Growth Rate YoY                 https://www.myfxbook.com/forex-economic-calendar/united-kingdom/gdp-growth-rate-yoy  Thu, 22 Dec 2022 07:00 GMT  medium       4.0%      2.4%       1.9%
15                  GDP Growth Rate QoQ                 https://www.myfxbook.com/forex-economic-calendar/united-kingdom/gdp-growth-rate-qoq  Thu, 22 Dec 2022 07:00 GMT  medium       0.1%     -0.2%      -0.3%
17               Interest Rate Decision                   https://www.myfxbook.com/forex-economic-calendar/indonesia/interest-rate-decision  Thu, 22 Dec 2022 07:30 GMT 